# Extended RNA-Seq Analysis Training Demo

## Overview

This tutorial demonstrates how to analyze a complete bulk RNA-Seq analysis zebrafish (*Danio rerio*) data set using the complete set of reads from the NCBI Sequence Read Archive and the entire genome assembly. Steps in the workflow include read trimming, read QC, read mapping, and counting mapped reads per gene to estimate gene expression. The tutorial also includes some extra steps, such as how to access SRA metadata using Athena, a more powerful method for retrieving reads compared to sra-tools. This tutorial workflow uses a subset of sequence reads from a study published by Hartig et al (PubMed ID: [2746894](https://pubmed.ncbi.nlm.nih.gov/32746894/); Gene Expression Omnibus: [GSE137987](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE137987)). In this study, the authors were interested in examining how chronic early life stress alters gene expression in adulthood. RNA samples were extracted from the blood of adult zebrafish who were exposed to cortisol or control conditions during just the first 5 days of development. There were three control (DMSO-treated) samples, and three cortisol-treated samples.

WARNING: This tutorial will take over 13 hours to run using ml.m5.4xlarge instance. The shorter version of this tutorial only analyzes a subset of the read data (1 million reads per sample) from this experiment and aligns them to just chromosome 14 rather than the whole genome because analyzing the entire data set takes much longer to run.

The read counts generated by this tutorial are the same as what is used in the R-based differential gene analysis workflow tutorial (Tutorial 2).

![RNA-Seq workflow](../../images/Tutorial_1_Workflow.png)

## STEP 1: Install Miniforge

First install Miniforge.

In [ ]:
# Download Miniforge or Mambaforge (you can use either based on preference)
!curl -L -O https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-$(uname)-$(uname -m).sh

# Install Miniforge (or Mambaforge) - no need to install conda since mamba will be available immediately
!bash Miniforge3-$(uname)-$(uname -m).sh -b -u -p $HOME/miniforge > /dev/null
!date +"%T"

Next, using mambaforge and bioconda, install the tools that will be used in this tutorial.

In [ ]:
# Update PATH to point to the Miniforge (or Mambaforge) bin files
import os
os.environ["PATH"] += os.pathsep + os.environ["HOME"]+"/miniforge/bin"

#now we can easily use 'mamba' command to install software 
!mamba install -y -c conda-forge -c bioconda trimmomatic fastqc multiqc sql-magic entrez-direct gffread parallel-fastq-dump sra-tools sql-magic pyathena samtools star rsem entrez-direct subread pigz -y > /dev/null

## STEP 2: Setup Environment

Create a set of directories in the sra-data-athena to store the reads, reference sequence files, and output files. Notice that first we remove the `data` directory to clean up files from Tutorial_1

In [ ]:
!cd $HOMEDIR
!echo $PWD
!mkdir -p data
!mkdir -p data/trunc_rawfastq
!mkdir -p data/trimmed
!mkdir -p data/fastqc/
!mkdir -p data/fastqc_samples/
!mkdir -p data/reference
!mkdir -p data/aligned_bam
!mkdir -p data/rsem_reference/zebrafish_rsem_reference
!mkdir -p data/rsem_output
!mkdir -p data/reference/STAR_index

Set # THREADS depending on your VM size

In [ ]:
import multiprocessing

num_cores = multiprocessing.cpu_count()
THREADS = max(1, num_cores - 1)

print("Number of threads:", THREADS)
os.environ["THREADS"] = str(THREADS)

## STEP 3: Downloading relevant FASTQ files using SRA Tools



Next we will need to download the relevant fastq files.

Because these files can be large, the process of downloading and extracting fastq files can be quite lengthy.

We will be downloading the sample runs from this project using SRA tools, downloading from the NCBI's SRA (Sequence Run Archives).

However, first we need to find the associated accession numbers in order to download.

### STEP 3.1: Finding run accession numbers.


The SRA stores sequence data in terms of runs, (SRR stands for Sequence Read Run). To download runs, we will need the accession ID for each run we wish to download.

The Mittenbühler MJ et al., project contains 8 runs. To make it easier, these are the run IDs associated with this project:

- SRR3371417
- SRR3371418
- SRR3371419
- SRR3371420
- SRR3371421
- SRR3371422

In this case, all these runs belong to the Bioproject PRJNA318296.

Sequence run experiments can be searched for using the SRA database on the NCBI website; and article-specific sample run information can be found in the supplementary section of that article.

Once the accession numbers are located, one can make a text file containing the list of accession IDs however they like.

Once again, to make things easier, we have made a .txt with these IDs that you can simply download here:

In [ ]:
!esearch -db sra -query "PRJNA318296" | efetch -format runinfo | cut -d',' -f1 | tail -n +2 > accs.txt
!cat accs.txt

### STEP 3.2 Finding run accession numbers using Athena (Optional)

Athena is a serverless query engine that allows you to analyze data stored in Amazon S3 using standard SQL. It offers several advantages over traditional SRA tools, making it a more efficient and scalable solution for large-scale RNA-seq data analysis.

Using Athena to access metadata is optional, but allows you to query large SRA metadata directly from AWS without needing to download and process files locally, making it faster and more scalable.

In [ ]:
from pyathena import connect
import pandas as pd

# Use the correct argument name: s3_staging_dir
conn = connect(s3_staging_dir='s3://sra-data-athena/', region_name='us-east-1')

In [ ]:
import boto3

# Initialize the Glue client
glue_client = boto3.client('glue', region_name='us-east-1')

# Run the crawler
crawler_name = 'sra_crawler'  # Use your crawler's name
glue_client.start_crawler(Name=crawler_name)

print(f"Crawler {crawler_name} started.")

In [ ]:
query = """
SELECT *
FROM AwsDataCatalog.srametadata.metadata
WHERE bioproject = 'PRJNA318296'
"""
df = pd.read_sql(query, conn)
df


In [ ]:
#write the SRR column to a text file
with open('accs.txt', 'w') as f:
    accs = df['acc'].to_string(header=False, index=False)
    f.write(accs)
    
#print the text file
!cat accs.txt

### STEP 3.3 Using the SRA-toolkit for a single sample.

The code snippet demonstrates how to download and preprocess single SRA data using the prefetch and fasterq-dump commands. First, prefetch downloads the specified SRA file and then, fasterq-dump converts the SRA file into paired-end FASTQ files. Finally, the generated FASTQ files are compressed using pigz to save space.

In [ ]:
# Example usage for SRA download:
!prefetch SRR3371420 -O data/raw_fastq -f yes

In [ ]:
#convert sra to fastq
!fasterq-dump data/raw_fastq/SRR3371420 -f -O data/raw_fastq/ -e $THREADS
#compress fastq to fastq.gz to save space
!pigz -p $THREADS data/raw_fastq/SRR3371420_1.fastq
!pigz -p $THREADS data/raw_fastq/SRR3371420_2.fastq

### STEP 3.4 Downloading multiple files using the SRA-toolkit.

The code uses prefetch to download multiple SRA files in parallel. It reads the list of SRR IDs from accs.txt, uses xargs to execute prefetch for each ID, and specifies the output directory and the -f option to create FASTQ files in the same directory as the SRA files. To speed up the download the code uses -P $THREADS option allowing parallel execution using the specified number of threads.

In [ ]:
!cat accs.txt | xargs -P $THREADS -I {} prefetch {} -O data/raw_fastq -f yes

### STEP 3.5 Converting Multiple SRA files to Fastq


In this step, the SRA files will be processed in parallel using parallel-fastq-dump. Each SRR ID from accs.txt will be read, and xargs will be used to execute parallel-fastq-dump for each SRA ID. This will result in the creation of two paired-end FASTQ files for each SRR ID, which will be compressed into a .gz file to save space.

In [ ]:
#!for x in `cat accs.txt`; do fasterq-dump -f -O data/raw_fastq -e $THREADS -m 4G data/raw_fastq/$x/$x.sra; done

##example of how to alternatively do the above process with parallel-fastq-dump using piping
!cat accs.txt | xargs -I {} parallel-fastq-dump -O data/raw_fastq/ --tmpdir . --threads $THREADS --gzip --split-files --sra-id {}

As before, it is good practice to turn .fastq files into .fastq.gz files to save space.

In our case, we will actually need to concatenate the fastq files later on, and so will zip after this.

The no redundant SRA files can also be deleted to save more space.

In [ ]:
#find and delete all SRR subfolders in the raw_fastq directory
!find data/raw_fastq -type d -name 'SRR*' -exec rm -rf {} \;

### STEP 3.6 Download reference transcriptome files that will be used by STAR


This step downloads and prepares the reference data needed for your RNA-seq analysis. It retrieves three essential files:

- Zebrafish genome (Danio_rerio.GRCz11.dna.primary_assembly.fa.gz): This compressed FASTA file contains the complete Zebrafish genome sequence, that will be used as the reference for aligning your RNA-seq reads.
- Zebrafish gene annotations (Danio_rerio.GRCz11.104.gtf.gz): This compressed GTF file provides information about the genes and transcripts in the Zebrafish genome, including their locations and structures. This data will crucial for interpreting the aligned RNA-seq reads and understanding what genes are expressed in each.
- Zebrafish feature table (GCF_000002035.6_GRCz11_feature_table.txt.gz): This compressed table provides additional annotations for the Zebrafish genome features, potentially including information about gene functions and pathways. This step will further used to analyze the differential gene expression (DEG) analysis. 

In [ ]:
! wget ftp://ftp.ensembl.org/pub/release-104/fasta/danio_rerio/dna/Danio_rerio.GRCz11.dna.primary_assembly.fa.gz -O data/reference/zebrafish_genome.fa.gz
! wget ftp://ftp.ensembl.org/pub/release-104/gtf/danio_rerio/Danio_rerio.GRCz11.104.gtf.gz -O data/reference/zebrafish_annotation.gtf.gz
! wget -O data/reference/zebrafish_feature_table.txt.gz https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/002/035/GCF_000002035.6_GRCz11/GCF_000002035.6_GRCz11_feature_table.txt.gz

In [ ]:
!gunzip -f data/reference/zebrafish_genome.fa.gz 
!gunzip -f data/reference/zebrafish_annotation.gtf.gz
!gunzip -f data/reference/zebrafish_feature_table.txt.gz

### STEP 3.7: Copy data file for Trimmomatic

One of trimmomatics functions is to trim sequence machine specific adapter sequences. These are usually within the trimmomatic installation directory in a folder called adapters.

Directories of packages within conda installations can be confusing, so in the case of using conda with trimmomatic, it may be easier to simply download or create a file with the relevant adapter sequencecs and store it in an easy to find directory.

In [ ]:
!wget -P data/trimmed/ https://sra-data-athena.s3.amazonaws.com/reference/TruSeq3-PE.fa

## STEP 4: Run FastQC

FastQC is an invaluable tool that allows you to evaluate whether there are problems with a set of reads. For example, it will provide a report of whether there is any bias in the sequence composition of the reads.

The below code may take a while to run. To make it run faster we can use threads to speed up the process.

In [ ]:
# Run fastqc for forward reads in parallel
!cat accs.txt | xargs -P $THREADS -I {} fastqc "data/raw_fastq/{}_1.fastq.gz" -o data/fastqc/

# Run fastqc for reverse reads in parallel
!cat accs.txt | xargs -P $THREADS -I {} fastqc "data/raw_fastq/{}_2.fastq.gz" -o data/fastqc/

Fastqc will output the results in HTML format, as below, for all forward and reverse reads.

In [ ]:
from IPython.display import IFrame
IFrame(src='./data/fastqc/SRR3371420_1_fastqc.html', width=800, height=600)

Although its best practice to look over them individually, tools like multiqc allow one to quickly look at a summary of the quality reports of the fastq files.

For instance, the below table shows which warnings, passes, or failures, from each fastqc report. There are other summaries created as well by multiqc.

In [ ]:
!multiqc -f data/fastqc/

import pandas as pd
dframe = pd.read_csv("./multiqc_data/multiqc_fastqc.txt", sep='\t')
display(dframe)

## STEP 5: Merging our fastq files (Optional if there are multiple SRR per GSM)

If the project used presents the multiple SRAs per GSM we can use Athena to access the SRA metadata and merging the FASTQ files, the code simplifies the subsequent analysis steps and reduces the number of files to process. This can improve efficiency and reduce computational overhead.
In this study this step was not used.

In [ ]:
from pyathena import connect
import pandas as pd

# Use the correct argument name: s3_staging_dir
conn = connect(s3_staging_dir='s3://sra-data-athena/', region_name='us-east-1')

query = """
SELECT *
FROM AwsDataCatalog.srametadata.metadata
WHERE bioproject = 'PRJNAXXXXXXX' #Change to the Bioproject number
AND organism = 'Danio rerio'
"""
df = pd.read_sql(
    query, conn
)
df

In [ ]:
#import os so we can easily pass strings to shell commands using 'subprocess'
import os
import subprocess

#now get the accession id's and sample id's from the created dataframe
runs = df['acc'].values
samples = list(set(df['acc'].values))

#sort them to be in numerical order
runs.sort()
samples.sort()
samples

In [ ]:
#now iterate through the samples, 
#because there are two SRRs to a run, 
#this means corresponding SRRs indices to an index of a GSM will be
#gsm index *2, and *2+1 
for index, item in enumerate(samples):
    
    #concatenate the two SRRs
    os.system(f"cat data/raw_fastq/{runs[index*2]}_1.fastq data/raw_fastq/{runs[index*2+1]}_1.fastq > data/raw_fastq/{samples[index]}_1.fastq")
    #delete the previous fastq files to save space
    os.system(f"rm data/raw_fastq/{runs[index*2]}_1.fastq")
    os.system(f"rm data/raw_fastq/{runs[index*2+1]}_1.fastq")
    #zip the merged fastq file to save more space
    os.system(f"gzip data/raw_fastq/{samples[index]}_1.fastq")
    
    #repeat for reverse reads
    os.system(f"cat data/raw_fastq/{runs[index*2]}_2.fastq data/raw_fastq/{runs[index*2+1]}_2.fastq > data/raw_fastq/{samples[index]}_2.fastq")
    
    os.system(f"rm data/raw_fastq/{runs[index*2]}_2.fastq")
    os.system(f"rm data/raw_fastq/{runs[index*2+1]}_2.fastq")  
   
    #its good practice to zip files to save space
    os.system(f"gzip data/raw_fastq/{samples[index]}_2.fastq")

In [ ]:
#since our files will now be samples, not SRRs we can write a new text file to use for downstream batch processes.
#we can use the DF we made in the previous cell.
with open('samples.txt', 'w') as f:
    df = df.sort_values(by='sample_name', ascending=True)
    samples = df['acc'].unique()
    samples = '\n'.join(map(str, samples))
    f.write(samples)
    
!cat samples.txt

## STEP 6: Run Trimmomatic

Trimmomatic will trim off any adapter sequences or low quality sequence it detects in the FASTQ files.

Using piping and our original list, it is possible to queue up a batch run of trimmomatic for all our files, note that this is a different way to run a loop compared with what we did before.

The below code may take approximately 30 minutes to run.

In [ ]:
!cat accs.txt | xargs -I {} \
trimmomatic PE -threads $THREADS \
'data/raw_fastq/{}_1.fastq.gz' 'data/raw_fastq/{}_2.fastq.gz' \
'data/trimmed/{}_1_trimmed.fastq' 'data/trimmed/{}_1_trimmed_unpaired.fastq' \
'data/trimmed/{}_2_trimmed.fastq' 'data/trimmed/{}_2_trimmed_unpaired.fastq' \
ILLUMINACLIP:data/trimmed/TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36

## STEP 7: Run FastQC
It's best practice to run FastQC after trimming. However, you may decide to run FastQC only once, before or after trimming.

We will proceed with only the forward reads -- this is because, looking at trimmomatic, there were very few 'orphaned' reads. That is to say, most forward and reverse reads were successfully paired together. Because we are just trying to map to a transcriptome, the read lengths of the forward reads alone, in this case, around 60 millions~ basepairs, should be sufficient.

The below code may take around 15-20 minutes to run.

In [ ]:
# Run FastQC
!cat accs.txt | xargs -P $THREADS -I {} fastqc data/trimmed/{}_1_trimmed.fastq data/trimmed/{}_2_trimmed.fastq -o data/fastqc_samples/

## STEP 8: Run MultiQC
MultiQC reads in the FastQC reports and generate a compiled report for all the analyzed FASTQ files.

In [ ]:
#!multiqc -f data/fastqc_samples/
!multiqc -f -o data/multiqc_samples/ data/fastqc_samples/

## STEP 9: Preparing the STAR-Compatible RSEM Reference

This command prepares a reference genome and annotation files for RNA-Seq analysis using RSEM (RNA-Seq by Expectation-Maximization) and STAR (Spliced Transcripts Alignment to a Reference). It generates files needed to quantify gene and isoform expression. The rsem-prepare-reference function takes a GTF file with gene annotations (zebrafish_annotation.gtf) and a FASTA file with the reference genome sequence (zebrafish_genome.fa). It processes these files to create a reference, saving the output in the zebrafish_reference directory. The --star option ensures the reference is compatible with STAR for efficient transcriptome alignment. The -p $THREADS option sets the number of threads used for parallel processing, speeding up the preparation process.

In [ ]:
# Preparing the reference genome
!rsem-prepare-reference --gtf data/reference/zebrafish_annotation.gtf --star -p $THREADS data/reference/zebrafish_genome.fa zebrafish_reference

## STEP 10: Run STAR for Alignment, Prepare and Run RSEM for Quantification

This script automates RNA-Seq gene expression quantification using RSEM and STAR. It reads SRR accession IDs from accs.txt, saves results in data/rsem_output, and runs rsem-calculate-expression for each ID. It uses paired-end trimmed FASTQ files from data/trimmed/ and a STAR-aligned RSEM reference (zebrafish_reference).

In [ ]:
import os

# Ensure you've set the path to the RSEM binary
# Read the SRR accessions from the file
with open('accs.txt', 'r') as f:
    srr_accessions = [line.strip() for line in f.readlines()]

# Define the output directory
output_dir = "data/rsem_output"

# Loop through each SRR accession and run rsem-calculate-expression
for srr in srr_accessions:
    !rsem-calculate-expression -p $THREADS --paired-end --star \
    data/trimmed/{srr}_1_trimmed.fastq data/trimmed/{srr}_2_trimmed.fastq zebrafish_reference data/rsem_output/{srr} > /dev/null

## STEP 11: Report the top 10 most highly expressed genes in the samples

Top 10 most highly expressed genes in each wild-type sample.


In [ ]:
import pandas as pd

# Path to RSEM results directory
rsem_results_dir = 'data/rsem_output'

# Loop through each file in accs.txt
for srr_id in open('accs.txt'):
    srr_id = srr_id.strip()  # Remove newline character
    rsem_result_file = f'{rsem_results_dir}/{srr_id}.genes.results'

    # Load the RSEM results into a Pandas DataFrame
    df = pd.read_csv(rsem_result_file, sep='\t')

    # Sort the DataFrame by TPM values in descending order and get the top 10 genes
    top_10_genes = df.sort_values(by='TPM', ascending=False).head(10)

    # Print the top 10 genes with their TPM values
    print(f"Top 10 Genes by TPM for {srr_id}:")
    print(top_10_genes[['gene_id', 'TPM']])

## STEP 12: Report the expression of irg1l for each file

Use `grep` to report the expression in the wild-type sample. The fields in the RSEM `genes.results` file are as follows. The level of expression is reported in the Transcripts Per Million (`TPM`) and number of reads (`NumReads`) fields:  
`Name    Length  EffectiveLength TPM     NumReads`

In [ ]:
import pandas as pd

# Path to RSEM results directory
rsem_results_dir = 'data/rsem_output'

# Target gene ID
target_gene = 'ENSDARG00000062788'

# Loop through each file in accs.txt
for srr_id in open('accs.txt'):
    srr_id = srr_id.strip()  # Remove newline character
    rsem_result_file = f'{rsem_results_dir}/{srr_id}.genes.results'

    # Load the RSEM results into a Pandas DataFrame
    df = pd.read_csv(rsem_result_file, sep='\t')

    # Filter for the target gene
    target_gene_data = df[df['gene_id'] == target_gene]

    # Print the target gene's TPM value for the SRR ID
    print(f"TPM for {target_gene} in {srr_id}: {target_gene_data['TPM'].values[0]}")

## STEP 13: Export Read counts to S3 Bucket


The code effectively extracts gene expression data from RSEM output files and stores them in a structured format on an S3 bucket. This data will be accessible for further analysis in Tutorial 2 and Tutorial 3.

In [ ]:
import os
import pandas as pd
import boto3

# Define the path to your RSEM output directory
rsem_output_path = "data/rsem_output"

# Define the S3 bucket and output path
s3_bucket = "sra-data-athena"
s3_output_path = "readcounts/"

# Initialize S3 client
s3_client = boto3.client('s3')

# Get a list of all .genes.results files in the directory
genes_files = [f for f in os.listdir(rsem_output_path) if f.endswith('.genes.results')]

# Loop through each file to extract gene ID, expected counts, and gene length
for file in genes_files:
    file_path = os.path.join(rsem_output_path, file)
    
    # Read the .genes.results file
    rsem_data = pd.read_csv(file_path, sep="\t")

    # Check if the necessary columns exist
    if all(col in rsem_data.columns for col in ["gene_id", "expected_count", "length"]):
        # Create a new dataframe with required columns
        result_data = rsem_data[["gene_id", "expected_count", "length"]]
        result_data.columns = ["GeneID", "Count", "GeneLength"]

        # Define the output filename based on the input file name
        output_file_name = f"{os.path.splitext(file)[0]}_zebrafish.txt"
        s3_output_file_path = f"{s3_output_path}{output_file_name}"

        # Convert the DataFrame to a CSV string
        csv_buffer = result_data.to_csv(sep="\t", index=False)

        # Upload the result directly to S3
        s3_client.put_object(Bucket=s3_bucket, Key=s3_output_file_path, Body=csv_buffer)

    else:
        print(f"Warning: Required columns are missing in file: {file}")

# Optionally, print a message indicating completion
print("Extraction and file creation complete.")

## STEP 14: Save Merged Read Counts

This code combines multiple RSEM gene count files into a single, unified file, making it easier to analyze and visualize the gene expression data. This files was also uploaded to S3 Bucket to allow further analysis in other Tutorials. 

In [ ]:
# Ensure the RSEM quantification results directory exists
!mkdir -p data/rsem_output

# Merge RSEM results by gene counts (similar to Salmon's numreads merge)
!rsem-generate-data-matrix data/rsem_output/*.genes.results > data/rsem_output/merged_gene_counts_zebrafish.txt

# Optionally, rename the columns based on the samples
# If you want to assign your GSM identifiers or any other custom names, edit the header.
!sed -i "1s/.*/Name\tGSM2121180\tGSM2121181\tGSM2121182\tGSM2121183\tGSM2121184\tGSM2121185/" data/rsem_output/merged_gene_counts_zebrafish.txt

# Remove any unnecessary prefixes like 'gene-' or 'rna-' for easier formatting
!sed -i "s/gene-//g" data/rsem_output/merged_gene_counts_zebrafish.txt
!sed -i "s/rna-//g" data/rsem_output/merged_gene_counts_zebrafish.txt

# Show a preview of the merged quantification file
!head data/rsem_output/merged_gene_counts_zebrafish.txt

import boto3
import os

# Define the file path and S3 bucket details
file_path = "data/rsem_output/merged_gene_counts_zebrafish.txt"
bucket_name = "sra-data-athena"
s3_key = "readcounts/merged_gene_counts_zebrafish.txt"

# Initialize an S3 client
s3_client = boto3.client('s3')

# Upload the file to the specified S3 bucket
try:
    s3_client.upload_file(file_path, bucket_name, s3_key)
    print(f"File {file_path} uploaded successfully to {bucket_name}/{s3_key}")
except Exception as e:
    print(f"Error uploading file: {e}")

# Define the file paths and S3 bucket details
rsem_output_path = "data/rsem_output"
feature_table_path = "data/reference/zebrafish_feature_table.txt"
bucket_name = "sra-data-athena"
s3_output_path = "readcounts/"
s3_feature_table_path = "reference/zebrafish_feature_table.txt"

# ... (rest of the code remains the same)

# Upload the gene count file
s3_client.upload_file(file_path, bucket_name, s3_key)

# Upload the feature table file
s3_client.upload_file(feature_table_path, bucket_name, s3_feature_table_path)

## <a name="workflow">Additional Workflows</a>

Now that you have read counts per gene, feel free to explore the R workflow which creates plots and analyses using these readcount files, or try other alternate workflows for creating read count files, such as using snakemake.


[Workflow One:](Tutorial_1_subsampling_zebrafish-miniforge.ipynb) A short introduction to downloading and mapping sequences to a zebrafish genome using STAR and RSEM.


[Workflow Two (DEG Analysis):](Tutorial_2_DEG_Analysis_zebrafish.ipynb) Using Deseq2 and R to conduct clustering and differential gene expression analysis.